In [0]:
loans_def_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.option("inferSchema", True) \
.load("/mnt/Lendingclub/Lendingclub/Lendingclub/raw/loans_defaulters_csv")

In [0]:
display(loans_def_raw_df.limit(10))

member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record
6d5091b3fcaaeb4eac37445042b6b79e8b21f16943b9c32c4f5dd74dcb0f2210,0.0,0.0,0.0,0.0,1.0,0.0,30.0,null
b5e7938b0a2da4ceaa75ea2e3c111d1e33ca0f944f954b6d65671ef395d21713,1.0,0.0,0.0,0.0,4.0,0.0,6.0,null
91060b858433e8a6107be9b76b324e304b5802631fc081b449188fe4b4e32205,0.0,0.0,0.0,0.0,0.0,0.0,null,null
cab1fa9f533688b0aab7c6be7b6256d15ad1dd5f3c4ca061a49cfeb26901977d,0.0,0.0,0.0,0.0,0.0,0.0,null,null
f74e401c1ab0adf788a7279dd70a27459fcc47733d35eed71149ff371d2a7283,1.0,0.0,0.0,0.0,3.0,0.0,12.0,null
8aef4bb29d609d8d684fd709af965bd690fc9ef3a2691777f38e0c10d622fe97,0.0,0.0,0.0,0.0,0.0,0.0,null,null
538b4653da3b1e8142b50baacf97a57b721eb6ec565f5fb63a1b71d191e9e934,0.0,0.0,0.0,0.0,0.0,0.0,49.0,null
b24d55f21390533c512f78f822153dc247e984dcd0cfe031c341ccef1c689817,1.0,0.0,0.0,0.0,0.0,0.0,3.0,null
1035c5401b0ca76d02c334b813302a9b72825e70b1659e8bc8ce5eea6c5d1f52,0.0,0.0,1.0,1.0,1.0,0.0,null,106.0
cb0f1777593e77909d0d359a6d9e633f28568500e17806f815395c92f0cd375d,0.0,0.0,0.0,0.0,0.0,0.0,75.0,null


In [0]:
loans_def_raw_df.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- delinq_amnt: double (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- pub_rec_bankruptcies: double (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- total_rec_late_fee: string (nullable = true)
 |-- mths_since_last_delinq: string (nullable = true)
 |-- mths_since_last_record: string (nullable = true)



In [0]:
loans_def_raw_df.createOrReplaceTempView("loan_defaulters")

In [0]:
spark.sql("select distinct(delinq_2yrs) from loan_defaulters")

DataFrame[delinq_2yrs: string]

In [0]:
display(spark.sql("select delinq_2yrs, count(*) as total from loan_defaulters group by delinq_2yrs order by total desc").limit(20))

delinq_2yrs,total
0.0,1838878
1.0,281335
2.0,81285
3.0,29539
4.0,13179
5.0,6599
6.0,3717
7.0,2062
8.0,1223
9.0,818


In [0]:
loan_defaulters_schema = "member_id string, delinq_2yrs float, delinq_amnt float, pub_rec float, pub_rec_bankruptcies float,inq_last_6mths float, total_rec_late_fee float, mths_since_last_delinq float, mths_since_last_record float"

In [0]:
loans_def_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.schema(loan_defaulters_schema) \
.load("/mnt/Lendingclub/Lendingclub/Lendingclub/raw/loans_defaulters_csv")

In [0]:
loans_def_raw_df.createOrReplaceTempView("loan_defaulters")

In [0]:
display(spark.sql("select delinq_2yrs, count(*) as total from loan_defaulters group by delinq_2yrs order by total desc").limit(10))

delinq_2yrs,total
0.0,1838878
1.0,281335
2.0,81285
3.0,29539
4.0,13179
5.0,6599
6.0,3717
7.0,2062
8.0,1223
9.0,818


In [0]:
from pyspark.sql.functions import col

In [0]:
loans_def_processed_df = loans_def_raw_df.withColumn("delinq_2yrs", col("delinq_2yrs").cast("integer")).fillna(0, subset = ["delinq_2yrs"])

In [0]:
loans_def_processed_df.createOrReplaceTempView("loan_defaulters")

In [0]:
display(spark.sql("select count(*) from loan_defaulters where delinq_2yrs is null"))

count(1)
0


In [0]:
display(spark.sql("select delinq_2yrs, count(*) as total from loan_defaulters group by delinq_2yrs order by total desc").limit(10))

delinq_2yrs,total
0,1839141
1,281337
2,81285
3,29545
4,13180
5,6601
6,3719
7,2063
8,1226
9,821


In [0]:
loans_def_delinq_df = spark.sql("select member_id,delinq_2yrs, delinq_amnt, int(mths_since_last_delinq) from loan_defaulters where delinq_2yrs > 0 or mths_since_last_delinq > 0")

In [0]:
display(loans_def_delinq_df.limit(10).limit(10))

member_id,delinq_2yrs,delinq_amnt,mths_since_last_delinq
6d5091b3fcaaeb4eac37445042b6b79e8b21f16943b9c32c4f5dd74dcb0f2210,0,0.0,30
b5e7938b0a2da4ceaa75ea2e3c111d1e33ca0f944f954b6d65671ef395d21713,1,0.0,6
f74e401c1ab0adf788a7279dd70a27459fcc47733d35eed71149ff371d2a7283,1,0.0,12
538b4653da3b1e8142b50baacf97a57b721eb6ec565f5fb63a1b71d191e9e934,0,0.0,49
b24d55f21390533c512f78f822153dc247e984dcd0cfe031c341ccef1c689817,1,0.0,3
cb0f1777593e77909d0d359a6d9e633f28568500e17806f815395c92f0cd375d,0,0.0,75
a962f4d59caec5fa1d51d5ad6ccf761573b476dbc25295e9645a491fa2bc6997,0,0.0,54
23857480ccf555ce4c5b7db19337de8cf8dd9eace1276036eb8f233a2c08cc6b,0,0.0,42
f2c4010f700d8c9c4433561b8fc9211a7bf6eefbabaee284c7cad51d64831e30,0,0.0,29
0d3c568ff6944b11c241b04737d140b1e6962491dfbd157774ca709c051acfc5,0,0.0,33


In [0]:
loans_def_delinq_df.count()

1106163

In [0]:
loans_def_records_enq_df = spark.sql("select member_id from loan_defaulters where pub_rec > 0.0 or pub_rec_bankruptcies > 0.0 or inq_last_6mths > 0.0")

In [0]:
display(loans_def_records_enq_df.limit(10))

member_id
6d5091b3fcaaeb4eac37445042b6b79e8b21f16943b9c32c4f5dd74dcb0f2210
b5e7938b0a2da4ceaa75ea2e3c111d1e33ca0f944f954b6d65671ef395d21713
f74e401c1ab0adf788a7279dd70a27459fcc47733d35eed71149ff371d2a7283
1035c5401b0ca76d02c334b813302a9b72825e70b1659e8bc8ce5eea6c5d1f52
9fe2d59ddf2a4f37e54e225954d87e21054e9c965b6d86f7bba4bf73e364e963
08bf9e080503b0113535d545974cb681ca3368ea42d23705cf961f9f56f55a8e
05ad4aed7c393035e6d29619d4bbb81164b0384f3b2e03876bf9223a1700bf70
0d3c568ff6944b11c241b04737d140b1e6962491dfbd157774ca709c051acfc5
18889919d4248ff4846801a65f3b7893da9621cd475fe9c1c91df8c0cfd336f4
159b209fd25dff55bfcf9ef91cb511b2cf45cb152afbfe7401286bb8ddc9c26f


In [0]:
loans_def_records_enq_df.count()

1070125

In [0]:
loans_def_delinq_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/mnt/Lendingclub/Lendingclub/Lendingclub/cleaned/loans_defaulters_deling_parquet") \
.save()

In [0]:
loans_def_records_enq_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/mnt/Lendingclub/Lendingclub/Lendingclub/cleaned/loans_defaulters_records_enq_parquet") \
.save()

In [0]:
loans_def_p_pub_rec_df = loans_def_processed_df.withColumn("pub_rec", col("pub_rec").cast("integer")).fillna(0, subset = ["pub_rec"])

In [0]:
loans_def_p_pub_rec_bankruptcies_df = loans_def_p_pub_rec_df.withColumn("pub_rec_bankruptcies", col("pub_rec_bankruptcies").cast("integer")).fillna(0, subset = ["pub_rec_bankruptcies"])

In [0]:
loans_def_p_inq_last_6mths_df = loans_def_p_pub_rec_bankruptcies_df.withColumn("inq_last_6mths", col("inq_last_6mths").cast("integer")).fillna(0, subset = ["inq_last_6mths"])

In [0]:
loans_def_p_inq_last_6mths_df.createOrReplaceTempView("loan_defaulters")

In [0]:
loans_def_detail_records_enq_df = spark.sql("select member_id, pub_rec, pub_rec_bankruptcies, inq_last_6mths from loan_defaulters")

In [0]:
display(loans_def_detail_records_enq_df.limit(10))

member_id,pub_rec,pub_rec_bankruptcies,inq_last_6mths
6d5091b3fcaaeb4eac37445042b6b79e8b21f16943b9c32c4f5dd74dcb0f2210,0,0,1
b5e7938b0a2da4ceaa75ea2e3c111d1e33ca0f944f954b6d65671ef395d21713,0,0,4
91060b858433e8a6107be9b76b324e304b5802631fc081b449188fe4b4e32205,0,0,0
cab1fa9f533688b0aab7c6be7b6256d15ad1dd5f3c4ca061a49cfeb26901977d,0,0,0
f74e401c1ab0adf788a7279dd70a27459fcc47733d35eed71149ff371d2a7283,0,0,3
8aef4bb29d609d8d684fd709af965bd690fc9ef3a2691777f38e0c10d622fe97,0,0,0
538b4653da3b1e8142b50baacf97a57b721eb6ec565f5fb63a1b71d191e9e934,0,0,0
b24d55f21390533c512f78f822153dc247e984dcd0cfe031c341ccef1c689817,0,0,0
1035c5401b0ca76d02c334b813302a9b72825e70b1659e8bc8ce5eea6c5d1f52,1,1,1
cb0f1777593e77909d0d359a6d9e633f28568500e17806f815395c92f0cd375d,0,0,0


In [0]:
loans_def_detail_records_enq_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/mnt/Lendingclub/Lendingclub/Lendingclub/cleaned/loans_def_detail_records_enq_df_parquet") \
.save()